## Dependencies

In [3]:
import importlib
import os
import pandas as pd
import sys

sys.path.append('../src/rag/')

import vector_store
importlib.reload(vector_store)

from vector_store import build_chroma_vector_store_from_df
from chromadb import PersistentClient

print("Libraries imported successfully!")

Libraries imported successfully!


In [ ]:
!pip uninstall -y keras keras-nightly keras-preprocessing tensorflow tensorflow-estimator
!pip install torch sentence-transformers
!pip install chromadb sentence-transformers

Found existing installation: keras 3.9.0
Uninstalling keras-3.9.0:
  Successfully uninstalled keras-3.9.0
Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [5]:
data_processed_folder = '../data/processed'
os.makedirs(data_processed_folder, exist_ok=True)

chroma_store_folder = '../data/store'
os.makedirs(chroma_store_folder, exist_ok=True)

## Load Full Instructed Dataset

In [6]:
filename = 'hyrox_full_data_instructed_feedback.csv'
file_path = os.path.join(data_processed_folder, filename)
df = pd.read_csv(file_path)
print(f"Data shape: {df.shape}")

Data shape: (166521, 126)


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166626 entries, 0 to 166625
Data columns (total 96 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   id                            166626 non-null  object 
 1   event_name                    166626 non-null  object 
 2   gender                        166626 non-null  int64  
 3   age_min                       166521 non-null  float64
 4   age_max                       166521 non-null  float64
 5   predicted_total_time          166626 non-null  float64
 6   residual                      166626 non-null  float64
 7   total_time                    166626 non-null  float64
 8   work_time                     166626 non-null  float64
 9   roxzone_time                  166626 non-null  float64
 10  run_time                      166626 non-null  float64
 11  run_1                         166626 non-null  float64
 12  1000m Ski                     166626 non-nul

In [57]:
df.tail(1)

,id,event_name,gender,age_min,age_max,predicted_total_time,residual,total_time,work_time,roxzone_time,run_time,run_1,1000m Ski,roxzone_1,run_2,50m Sled Push,roxzone_2,run_3,50m Sled Pull,roxzone_3,run_4,80m Burpee Broad Jump,roxzone_4,run_5,1000m Row,roxzone_5,run_6,200m Farmer Carry,roxzone_6,run_7,100m Sandbag Lunges,roxzone_7,run_8,100 Wall Balls,total_time_zscore,work_time_zscore,roxzone_time_zscore,run_time_zscore,run_1_zscore,1000m Ski_zscore,roxzone_1_zscore,run_2_zscore,50m Sled Push_zscore,roxzone_2_zscore,run_3_zscore,50m Sled Pull_zscore,roxzone_3_zscore,run_4_zscore,80m Burpee Broad Jump_zscore,roxzone_4_zscore,run_5_zscore,1000m Row_zscore,roxzone_5_zscore,run_6_zscore,200m Farmer Carry_zscore,roxzone_6_zscore,run_7_zscore,100m Sandbag Lunges_zscore,roxzone_7_zscore,run_8_zscore,100 Wall Balls_zscore,cluster_perf_only,cluster_perf_context,total_time_minmax,work_time_minmax,roxzone_time_minmax,run_time_minmax,run_1_minmax,1000m Ski_minmax,roxzone_1_minmax,run_2_minmax,50m Sled Push_minmax,roxzone_2_minmax,run_3_minmax,50m Sled Pull_minmax,roxzone_3_minmax,run_4_minmax,80m Burpee Broad Jump_minmax,roxzone_4_minmax,run_5_minmax,1000m Row_minmax,roxzone_5_minmax,run_6_minmax,200m Farmer Carry_minmax,roxzone_6_minmax,run_7_minmax,100m Sandbag Lunges_minmax,roxzone_7_minmax,run_8_minmax,100 Wall Balls_minmax,suggestions_perf_only,suggestions_perf_context,performance_feedback,input,context,response
166625,"124015_Kinkopf, Heike_GER",S4 2021 Stuttgart,1,50.0,54.0,130.529551,0.387116,130.916667,52.883333,10.4,67.633333,7.266667,5.716667,0.533333,7.55,2.483333,1.516667,7.766667,6.233333,1.383333,7.983333,12.583333,2.383333,9.35,5.683333,1.366667,8.366667,2.266667,1.45,8.5,10.866667,1.766667,10.85,7.05,1.592881,1.350129,1.03365,1.759755,1.56061,1.03411,-0.371672,1.549176,-0.276795,1.131029,1.318311,0.522627,0.813529,1.441389,2.607203,1.83713,1.945994,0.676047,0.779327,1.590983,0.145823,0.334022,1.656163,2.678968,0.473995,1.876982,0.297506,2,2,0.510695,0.401341,0.157417,0.454984,0.152929,0.511177,0.020038,0.173563,0.046403,0.083029,0.202433,0.107937,0.048005,0.250785,0.281297,0.076348,0.216853,0.37146,0.045505,0.244282,0.075055,0.054206,0.191585,0.198237,0.031122,0.184996,0.125855,"['run_1', '1000m Ski', 'run_2', '80m Burpee Broad Jump', 'roxzone_4', 'run_5', 'run_7', '100m Sandbag Lunges', 'run_8']","['run_1', '1000m Ski', 'run_2', '80m Burpee Broad Jump', 'roxzone_4', 'run_5', 'run_6', 'run_7', '100m Sandbag Lunges', 'run_8']","You performed close to expected levels. Compared to all athletes, you could improve: run_1, 1000m Ski, run_2, 80m Burpee Broad Jump, roxzone_4, run_5, run_7, 100m Sandbag Lunges, run_8. Compared to athletes of similar age and gender, your weak areas were: run_1, 1000m Ski, run_2, 80m Burpee Broad Jump, roxzone_4, run_5, run_6, run_7, 100m Sandbag Lunges, run_8.",Generate personalized feedback for a HYROX Open Division participant based on their performance.,"gender: 1, age_range: 50.0-54.0, total_time: 130, predicted_time: 131, residual: 0, cluster_perf_only: 2, cluster_perf_context: 2","You performed close to expected levels. Compared to all athletes, you could improve: run_1, 1000m Ski, run_2, 80m Burpee Broad Jump, roxzone_4, run_5, run_7, 100m Sandbag Lunges, run_8. Compared to athletes of similar age and gender, your weak areas were: run_1, 1000m Ski, run_2, 80m Burpee Broad Jump, roxzone_4, run_5, run_6, run_7, 100m Sandbag Lunges, run_8."


**Make sure there are no duplicate entries**

In [58]:
df[['id', 'event_name']].duplicated().sum()

0

In [59]:
df[['age_min', 'age_max', 'gender', 'input', 'context', 'response']].isna().sum()

age_min     105
age_max     105
gender        0
input         0
context       0
response      0
dtype: int64

In [60]:
df = df.dropna(subset=['age_min', 'age_max'])

## Build the Vector Store

In [32]:
# define the columns to be used for embeddings, metadata, and documents
embedding_cols = [
    "run_1", "run_1_minmax", "run_1_zscore", "1000m Ski", "1000m Ski_minmax", "1000m Ski_zscore", "roxzone_1", "roxzone_1_minmax", "roxzone_1_zscore",
    "run_2", "run_2_minmax", "run_2_zscore", "50m Sled Push", "50m Sled Push_minmax", "50m Sled Push_zscore", "roxzone_2", "roxzone_2_minmax", "roxzone_2_zscore",
    "run_3", "run_3_minmax", "run_3_zscore", "50m Sled Pull", "50m Sled Pull_minmax", "50m Sled Pull_zscore", "roxzone_3", "roxzone_3_minmax", "roxzone_3_zscore",
    "run_4", "run_4_minmax", "run_4_zscore", "80m Burpee Broad Jump", "80m Burpee Broad Jump_minmax", "80m Burpee Broad Jump_zscore", "roxzone_4", "roxzone_4_minmax", "roxzone_4_zscore",
    "run_5", "run_5_minmax", "run_5_zscore", "1000m Row", "1000m Row_minmax", "1000m Row_zscore", "roxzone_5", "roxzone_5_minmax", "roxzone_5_zscore",
    "run_6", "run_6_minmax", "run_6_zscore", "200m Farmer Carry", "200m Farmer Carry_minmax", "200m Farmer Carry_zscore", "roxzone_6", "roxzone_6_minmax", "roxzone_6_zscore",
    "run_7", "run_7_minmax", "run_7_zscore", "100m Sandbag Lunges", "100m Sandbag Lunges_minmax", "100m Sandbag Lunges_zscore", "roxzone_7", "roxzone_7_minmax", "roxzone_7_zscore",
    "run_8", "run_8_minmax", "run_8_zscore", "100 Wall Balls", "100 Wall Balls_minmax", "100 Wall Balls_zscore",
    "run_time", "run_time_minmax", "run_time_zscore",
    "work_time", "work_time_minmax", "work_time_zscore",
    "roxzone_time", "roxzone_time_minmax", "roxzone_time_zscore",
    "total_time", "total_time_minmax", "total_time_zscore",
    "predicted_total_time",
    "residual"
]
metadata_cols = ["event_name", "gender", "age_min", "age_max"]
document_cols = ["response"] # ["performance_feedback", "input", "context", "response"]
id_cols = ["id", "event_name"]

In [33]:
# preview sample of data to confirm it's ok
df[["id"]+ metadata_cols +  embedding_cols[:3] + document_cols].sample(1)

,id,event_name,gender,age_min,age_max,run_1,run_1_minmax,run_1_zscore,response
120464,"134001_Dainat, Stefan_GER",S7 2025 Karlsruhe,0,45.0,49.0,4.55,0.095756,-0.0346,"You performed close to expected levels. Compared to all athletes, you could improve: 80m Burpee Broad Jump, 100m Sandbag Lunges, 100 Wall Balls. Compared to athletes of similar age and gender, your weak areas were: 80m Burpee Broad Jump, 100m Sandbag Lunges, 100 Wall Balls."


### ChromaDB Vector Store

In [26]:
chroma_store_folder = '../data/store'
os.makedirs(chroma_store_folder, exist_ok=True)

In [34]:
build_chroma_vector_store_from_df(
    df=df,
    embedding_cols=embedding_cols,
    metadata_cols=metadata_cols,
    document_cols=document_cols,
    id_cols=id_cols,
    persist_dir=chroma_store_folder,
    debug_mode=True
)

Batches:   0%|          | 0/163 [00:00<?, ?it/s]

✅ Deleted existing collection 'hyrox_participants' (debug mode)
✅ Added batch 1 (5460 items)
✅ Added batch 2 (5460 items)
✅ Added batch 3 (5460 items)
✅ Added batch 4 (5460 items)
✅ Added batch 5 (5460 items)
✅ Added batch 6 (5460 items)
✅ Added batch 7 (5460 items)
✅ Added batch 8 (5460 items)
✅ Added batch 9 (5460 items)
✅ Added batch 10 (5460 items)
✅ Added batch 11 (5460 items)
✅ Added batch 12 (5460 items)
✅ Added batch 13 (5460 items)
✅ Added batch 14 (5460 items)
✅ Added batch 15 (5460 items)
✅ Added batch 16 (5460 items)
✅ Added batch 17 (5460 items)
✅ Added batch 18 (5460 items)
✅ Added batch 19 (5460 items)
✅ Added batch 20 (5460 items)
✅ Added batch 21 (5460 items)
✅ Added batch 22 (5460 items)
✅ Added batch 23 (5460 items)
✅ Added batch 24 (5460 items)
✅ Added batch 25 (5460 items)
✅ Added batch 26 (5460 items)
✅ Added batch 27 (5460 items)
✅ Added batch 28 (5460 items)
✅ Added batch 29 (5460 items)
✅ Added batch 30 (5460 items)
✅ Added batch 31 (2721 items)
✅ Vector store 

## Vector Store Query Testing

#### Load the vector store

In [61]:
collection_name = "hyrox_participants"
persist_dir = chroma_store_folder

# initialize ChromaDB
client = client = PersistentClient(path=persist_dir)

# get colletion
collection = client.get_collection(name=collection_name)

In [62]:
# example of results of hyrox race in minutes
race_minutes = {
    "run_1": 6.97,                  # 00:06:58
    "1000m Ski": 5.17,              # 00:05:10
    "roxzone_1": 0.25,              # 15 seg

    "run_2": 5.67,                  # 00:05:40
    "50m Sled Push": 3.33,          # 00:03:20
    "roxzone_2": 0.42,              # 25 seg

    "run_3": 6.08,                  # 00:06:05
    "50m Sled Pull": 8.20,          # 00:08:12
    "roxzone_3": 0.30,              # 18 seg

    "run_4": 6.47,                  # 00:06:28
    "80m Burpee Broad Jump": 9.70, # 00:09:42
    "roxzone_4": 0.48,              # 29 seg

    "run_5": 7.47,                  # 00:07:28
    "1000m Row": 5.57,             # 00:05:34
    "roxzone_5": 1.13,              # 68 seg

    "run_6": 7.48,                  # 00:07:29
    "200m Farmer Carry": 2.05,     # 00:02:03
    "roxzone_6": 0.53,              # 32 seg

    "run_7": 7.22,                  # 00:07:13
    "100m Sandbag Lunges": 5.20,   # 00:05:12
    "roxzone_7": 1.27,              # 76 seg

    "run_8": 8.25,                  # 00:08:15
    "100 Wall Balls": 5.20,        # 00:05:12

    "run_time": 55.53,             # 55:32
    "work_time": 44.42,            # 2665 seg
    "roxzone_time": 3.52,          # 211 seg
    "total_time": 74.92            # 1:14:55
}

#### Query 1

In [65]:
n_results = 5
query_text = f"I want to know the performance of { n_results } participants who performed similar to me in all races"

results = collection.query(
    query_texts=[query_text],
    n_results=n_results
)

for i, doc in enumerate(results["documents"][0]):
    print(f"🔹 Match #{i+1}")
    print(f"ID: {results['ids'][0][i]}")
    print(f"Distance: {results['distances'][0][i]:.4f}")
    print(f"Metadata: {results['metadatas'][0][i]}")
    print(f"Feedback: {doc}")
    print("-" * 60)

🔹 Match #1
ID: 150009_Mahboub, Sara_ITA_S7 2024 Milan
Distance: 1.5685
Metadata: {'age_min': 35.0, 'age_max': 39.0, 'event_name': 'S7 2024 Milan', 'gender': 1}
Feedback: You performed close to expected levels. Compared to all athletes, you could improve: run_1, 1000m Ski, roxzone_1, 50m Sled Push, roxzone_2, roxzone_3, 80m Burpee Broad Jump, 1000m Row, roxzone_6. Compared to athletes of similar age and gender, your weak areas were: run_1, 1000m Ski.
------------------------------------------------------------
🔹 Match #2
ID: 163025_Murtagh, Bella_GBR_S4 2022 London
Distance: 1.5713
Metadata: {'age_min': 25.0, 'gender': 1, 'event_name': 'S4 2022 London', 'age_max': 29.0}
Feedback: You performed close to expected levels. Compared to all athletes, you could improve: 1000m Ski, 50m Sled Push, roxzone_2, 50m Sled Pull, 200m Farmer Carry. Compared to athletes of similar age and gender, your weak areas were: 1000m Ski, 50m Sled Push, roxzone_2, 50m Sled Pull, 200m Farmer Carry.
---------------

#### Query 2

In [63]:
n_results = 1
query_text = "I want to find participants who had similar overall performance to me, but who did better in the Wall Balls and the Sled Pull"

results = collection.query(query_texts=[query_text], n_results=n_results)

In [66]:
df.tail(1)

,id,event_name,gender,age_min,age_max,predicted_total_time,residual,total_time,work_time,roxzone_time,run_time,run_1,1000m Ski,roxzone_1,run_2,50m Sled Push,roxzone_2,run_3,50m Sled Pull,roxzone_3,run_4,80m Burpee Broad Jump,roxzone_4,run_5,1000m Row,roxzone_5,run_6,200m Farmer Carry,roxzone_6,run_7,100m Sandbag Lunges,roxzone_7,run_8,100 Wall Balls,total_time_zscore,work_time_zscore,roxzone_time_zscore,run_time_zscore,run_1_zscore,1000m Ski_zscore,roxzone_1_zscore,run_2_zscore,50m Sled Push_zscore,roxzone_2_zscore,run_3_zscore,50m Sled Pull_zscore,roxzone_3_zscore,run_4_zscore,80m Burpee Broad Jump_zscore,roxzone_4_zscore,run_5_zscore,1000m Row_zscore,roxzone_5_zscore,run_6_zscore,200m Farmer Carry_zscore,roxzone_6_zscore,run_7_zscore,100m Sandbag Lunges_zscore,roxzone_7_zscore,run_8_zscore,100 Wall Balls_zscore,cluster_perf_only,cluster_perf_context,total_time_minmax,work_time_minmax,roxzone_time_minmax,run_time_minmax,run_1_minmax,1000m Ski_minmax,roxzone_1_minmax,run_2_minmax,50m Sled Push_minmax,roxzone_2_minmax,run_3_minmax,50m Sled Pull_minmax,roxzone_3_minmax,run_4_minmax,80m Burpee Broad Jump_minmax,roxzone_4_minmax,run_5_minmax,1000m Row_minmax,roxzone_5_minmax,run_6_minmax,200m Farmer Carry_minmax,roxzone_6_minmax,run_7_minmax,100m Sandbag Lunges_minmax,roxzone_7_minmax,run_8_minmax,100 Wall Balls_minmax,suggestions_perf_only,suggestions_perf_context,performance_feedback,input,context,response
166625,"124015_Kinkopf, Heike_GER",S4 2021 Stuttgart,1,50.0,54.0,130.529551,0.387116,130.916667,52.883333,10.4,67.633333,7.266667,5.716667,0.533333,7.55,2.483333,1.516667,7.766667,6.233333,1.383333,7.983333,12.583333,2.383333,9.35,5.683333,1.366667,8.366667,2.266667,1.45,8.5,10.866667,1.766667,10.85,7.05,1.592881,1.350129,1.03365,1.759755,1.56061,1.03411,-0.371672,1.549176,-0.276795,1.131029,1.318311,0.522627,0.813529,1.441389,2.607203,1.83713,1.945994,0.676047,0.779327,1.590983,0.145823,0.334022,1.656163,2.678968,0.473995,1.876982,0.297506,2,2,0.510695,0.401341,0.157417,0.454984,0.152929,0.511177,0.020038,0.173563,0.046403,0.083029,0.202433,0.107937,0.048005,0.250785,0.281297,0.076348,0.216853,0.37146,0.045505,0.244282,0.075055,0.054206,0.191585,0.198237,0.031122,0.184996,0.125855,"['run_1', '1000m Ski', 'run_2', '80m Burpee Broad Jump', 'roxzone_4', 'run_5', 'run_7', '100m Sandbag Lunges', 'run_8']","['run_1', '1000m Ski', 'run_2', '80m Burpee Broad Jump', 'roxzone_4', 'run_5', 'run_6', 'run_7', '100m Sandbag Lunges', 'run_8']","You performed close to expected levels. Compared to all athletes, you could improve: run_1, 1000m Ski, run_2, 80m Burpee Broad Jump, roxzone_4, run_5, run_7, 100m Sandbag Lunges, run_8. Compared to athletes of similar age and gender, your weak areas were: run_1, 1000m Ski, run_2, 80m Burpee Broad Jump, roxzone_4, run_5, run_6, run_7, 100m Sandbag Lunges, run_8.",Generate personalized feedback for a HYROX Open Division participant based on their performance.,"gender: 1, age_range: 50.0-54.0, total_time: 130, predicted_time: 131, residual: 0, cluster_perf_only: 2, cluster_perf_context: 2","You performed close to expected levels. Compared to all athletes, you could improve: run_1, 1000m Ski, run_2, 80m Burpee Broad Jump, roxzone_4, run_5, run_7, 100m Sandbag Lunges, run_8. Compared to athletes of similar age and gender, your weak areas were: run_1, 1000m Ski, run_2, 80m Burpee Broad Jump, roxzone_4, run_5, run_6, run_7, 100m Sandbag Lunges, run_8."
